In [1]:
!pip install pytube
!pip install openai-whisper
!pip install langchain
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 13.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 68.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803319 sha256=4f5fb952714a323c0890b3881ef935f29b58c624e7ff3262091b929658fa99f1
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 

In [2]:
from torch import cuda, torch
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [3]:
from pytube import YouTube

def download_audio_from_youtube(url):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_path = audio_stream.download(filename="audio.mp3")
    return audio_path


In [4]:
import whisper

def transcribe_audio_with_whisper(audio_path):
    model = whisper.load_model("base")  # or "small", "medium", "large" depending on needs
    transcription = model.transcribe(audio_path)
    return transcription['text']


In [5]:
from langchain.tools import Tool

def youtube_to_text_tool(url):
    audio_path = download_audio_from_youtube(url)
    transcription = transcribe_audio_with_whisper(audio_path)
    return transcription

youtube_transcription_tool = Tool(
    name="YouTubeTranscriptionTool",
    func=youtube_to_text_tool,
    description="Converts YouTube video audio to text."
)

In [6]:
!pip install python-dotenv

In [7]:
from dotenv import load_dotenv, find_dotenv
import getpass
import os

# Load Environment Variable
_ = load_dotenv(find_dotenv())

# Set the environment variable directly in the script
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
# initialize connection to pinecone (get API key at app.pinecone.io)
#PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") # or "YOUR_API_KEY"

In [9]:
# Access the environment variable with os.environ
# print(os.environ["OPENAI_API_KEY"])

In [10]:
""" !pip install --upgrade pytube
!pip install yt-dlp
!pip install imageio[ffmpeg] """


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 89.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 137.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
   ━━━━━

In [31]:
import os
print(os.environ["PATH"])


/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/root/.local/bin:/usr/local/cuda/bin


In [11]:
import subprocess
subprocess.run(["ffmpeg", "-version"])


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

CompletedProcess(args=['ffmpeg', '-version'], returncode=0)

In [33]:
# Not Required
#!apt-get update && apt-get install -y ffmpeg

#!which ffmpeg

/usr/bin/ffmpeg


In [12]:
# to run it on PAPERSPACE!

import yt_dlp

def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }
        ],
        'outtmpl': 'audio_accent_1hr',
        'ffmpeg_location': '/usr/bin/ffmpeg'  # Updated path to ffmpeg on Paperspace
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return 'audio_accent_1hr.mp3'

# Assuming transcribe_audio_with_whisper is a function that takes an audio path and transcribes it with Whisper
video_url = "https://www.youtube.com/watch?v=rhiYfBctLOI"
audio_path = download_audio(video_url)
transcription = transcribe_audio_with_whisper(audio_path)
#print(transcription)

[youtube] Extracting URL: https://www.youtube.com/watch?v=rhiYfBctLOI
[youtube] rhiYfBctLOI: Downloading webpage
[youtube] rhiYfBctLOI: Downloading ios player API JSON
[youtube] rhiYfBctLOI: Downloading mweb player API JSON
[youtube] rhiYfBctLOI: Downloading player 4e23410d
[youtube] rhiYfBctLOI: Downloading m3u8 information
[info] rhiYfBctLOI: Downloading 1 format(s): 251
[download] Destination: audio_accent_1hr
[download] 100% of   51.43MiB in 00:00:01 at 30.88MiB/s    
[ExtractAudio] Destination: audio_accent_1hr.mp3
Deleting original file audio_accent_1hr (pass -k to keep)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 179MiB/s]


In [13]:
# Save transcription to HTML
with open('transcription.html', 'w', encoding='utf-8') as file:
    file.write("<html><body><h1>Transcription</h1><ol>")
    for line in transcription.splitlines():  # Split by lines if it's a long string
        file.write(f"<li>{line}</li>")
    file.write("</ol></body></html>")

print("Transcription saved as HTML.")

Transcription saved as HTML.


In [17]:
#!pip install -U langchain-community

In [18]:
# SEGMENT and VECTORIZE the Transcription with Langchain

from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the OpenAI embedding model 
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=400)
segments = text_splitter.split_text(transcription)

# Vectorize each segment and store in a list with segment metadata
vectorized_segments = []
embeddings = embedding_model.embed_documents(segments)  # Generate embeddings for all segments

for i, (segment, vector) in enumerate(zip(segments, embeddings)):
    vectorized_segments.append({
        "id": f"segment_{i}",
        "text": segment,
        "embedding": vector
    })

print("Vectorization complete. Ready for storage in a vector database.")



Vectorization complete. Ready for storage in a vector database.


In [20]:
#!pip install pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 21.5 MB/s eta 0:00:00


In [23]:
from pinecone import Pinecone

# Securely fetch OpenAI and Pinecone API keys                    
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass("Enter Pinecone API key: ")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("Enter OpenAI API key: ")

In [27]:
#print(os.environ["PINECONE_API_KEY"])

In [24]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

In [28]:
# EMBEDDINGS OpenAI-ada-002

import os
import time
from getpass import getpass
from tqdm.auto import tqdm
from langchain.embeddings import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec

# Securely fetch OpenAI and Pinecone API keys
#os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass("Enter OpenAI API key: ")
#os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass("Enter Pinecone API key: ")
#os.environ["PINECONE_ENV"] = os.getenv("PINECONE_ENV") or "us-west1-gcp"  # Example environment

# Initialize OpenAI embeddings model
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=os.environ["OPENAI_API_KEY"])

# Initialize Pinecone client with the updated API
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
#spec = ServerlessSpec(cloud="aws", region="us-west-1")  # Specify Pinecone environment
index_name = "transcription-qa-index"

# Check if the index exists; create if it doesn’t
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embedding dimension
        metric="dotproduct",
        spec=spec
    )
    # Wait for the index to be ready
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

# Connect to the index for upsertion
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [29]:
# Upsert embeddings to Pinecone in batches
batch_size = 50
for i in tqdm(range(0, len(vectorized_segments), batch_size)):
    batch = vectorized_segments[i:i + batch_size]
    ids = [segment["id"] for segment in batch]
    embeds = [segment["embedding"] for segment in batch]
    metadatas = [{"text": segment["text"]} for segment in batch]
    index.upsert(vectors=zip(ids, embeds, metadatas))

print("All segments upserted into Pinecone.")

  0%|          | 0/3 [00:00<?, ?it/s]

All segments upserted into Pinecone.


In [30]:
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 103}},
 'total_vector_count': 103}

In [32]:
# EMBED user QUERY and retrieve RELEVANT SEGMENTS (K-5)
# generate a SUMMARY at first # NEED TO relook at this requirement, as it is only appending the segments. or use GPT4>

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Step 1: Embed User Query and Retrieve Relevant Segments
def retrieve_relevant_segments(query):
    query_embedding = embedding_model.embed_query(query)
    results = index.query(vector=query_embedding, top_k=5, include_metadata=True)
    segments = [match['metadata']['text'] for match in results['matches']]
    return segments


# Step 2: Generate Summary from Retrieved Segments
def generate_summary(segments):
    # Concatenate segments to prepare for summarization
    context = " ".join(segments)
    summary_prompt = PromptTemplate(input_variables=["context"], template="Summarize the following: {context}")
    summary = summary_prompt.format(context=context)
    return summary

# Sample user query
user_query = "Tell me about this subject in simple terms."
segments = retrieve_relevant_segments(user_query)
summary = generate_summary(segments)

print("Summary of Subject:", summary)

Summary of Subject: Summarize the following: of having James with us representing the academic world. And what's going on in the research world? We want to learn from that as the industry. So if that makes sense, and we'll have a dialogue with all of you, we have a few comments from the colleagues here, and then we'll open it up for a good interaction. So with that, we'll go to you and you. Maybe you give us a little update on what's going on in the policy world. OK, thanks, Arnapp. So I was given, I think, five minutes to talk about the state of AI policy globally. So we'll see how to do that. When I thought I might do, I'll try to give a brief overview of sort of where things stand, the trends that are emerging. I'll do a minute or two on the EUA IAC, both because we're here, and for reasons I think it's particularly important for this discussion. And I'll touch on sort of where global governance is, and where it may be going. But just to say, for someone in sort of the policy world,

In [37]:
# AGENT setup for QA interaction:

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# Initialize memory to track conversation context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize the Chat model (e.g., GPT-3.5)
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])

# Wrap Pinecone index with LangChain's Pinecone vector store
vector_store = Pinecone(index=index, embedding=embedding_model, text_key="text")  # Specify "text" as the key for document content

# Create Conversational Retrieval Chain using the wrapped vector store and memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=chat_model,
    retriever=vector_store.as_retriever(),  # Use the vector store as the retriever
    memory=memory
)

# Define a function to handle the initial interaction
def initial_interaction(user_query):
    # Step 1: Retrieve and summarize
    segments = retrieve_relevant_segments(user_query)
    summary = generate_summary(segments)
    
    # Step 2: Present the summary to the user and prompt for follow-ups
    print("Summary of Subject:", summary)
    follow_up_query = input("Do you have any further questions on this topic? ")
    
    # Step 3: Run the follow-up query through the agent
    response = qa_chain({"question": follow_up_query})
    print("Agent's Response:", response['answer'])

# Example call to start the interaction
user_query = "Tell me about this subject in simple terms."
initial_interaction(user_query)


Summary of Subject: Summarize the following: of having James with us representing the academic world. And what's going on in the research world? We want to learn from that as the industry. So if that makes sense, and we'll have a dialogue with all of you, we have a few comments from the colleagues here, and then we'll open it up for a good interaction. So with that, we'll go to you and you. Maybe you give us a little update on what's going on in the policy world. OK, thanks, Arnapp. So I was given, I think, five minutes to talk about the state of AI policy globally. So we'll see how to do that. When I thought I might do, I'll try to give a brief overview of sort of where things stand, the trends that are emerging. I'll do a minute or two on the EUA IAC, both because we're here, and for reasons I think it's particularly important for this discussion. And I'll touch on sort of where global governance is, and where it may be going. But just to say, for someone in sort of the policy world,

In [36]:
from langchain.output_parsers import ResponseParser
from langchain.prompts import PromptTemplate

# Define a simple parser to format responses into paragraphs with bullet points
class BulletPointParser(ResponseParser):
    def parse(self, response: str) -> str:
        # Transform each sentence into a bullet point
        return "\n".join([f"- {sentence.strip()}" for sentence in response.split(". ") if sentence])

# Define the response prompt with improved formatting
response_template = PromptTemplate(
    template="Answer the question in bullet points: {question}"
)

# Initialize a formatted conversational loop
def interactive_chat():
    print("Welcome to the Q&A Chatbot! Ask me anything.")
    while True:
        user_query = input("You: ")
        if user_query.lower() == "no":
            print("Agent: Thank you for using the Q&A chatbot. Goodbye!")
            break
        response = qa_chain({"question": user_query})
        # Parse and format the response
        formatted_response = BulletPointParser().parse(response['answer'])
        print("Agent's Response:\n", formatted_response)
        print("\nAgent: Do you have more questions?")

# Start the interactive chat
interactive_chat()


ImportError: cannot import name 'ResponseParser' from 'langchain.output_parsers' (/usr/local/lib/python3.11/dist-packages/langchain/output_parsers/__init__.py)

In [21]:
# TAMIL TRANSCRIPTION GENERATION 


""" from whisper import load_model

# Load the Whisper model
model1 = load_model("large")  # or use "small" or "base" for faster processing with lower accuracy


# Transcribe the Tamil audio file
result = model1.transcribe("audio_tamil.mp3", language="ta")
transcription = result["text"]

print(transcription) """

100%|█████████████████████████████████████| 2.88G/2.88G [00:53<00:00, 57.4MiB/s]


 இப்போது புதிய அறிமுகம் அச்சி ஹோட்டல் சாம்பார் 50 கிராம் ரும்பாய் 20 மட்டுமே சென்னை பெருநகர் மற்றும் புரனகரில் சுமார் ஒரு மணி நேரமாக வெளித்து வாங்கும் கணமழை ராயப் பேட்டை, தியாகராய நகர், எழும்பூர், அண்ணாசாலை, அண்ணா நகர் meantime anchor உள்ளிட்ட பகுதிகளில் இடி மின்னவுடன் கொட்டி தீர்க்கும் கணமழை சென்னையில் மேற்கு அண்ணா நகர் பகுதியில் ஒரு மணி நேரத்துயில் 90 cómbing kindergarten அளவிருக்கு கணமழை பதிபு கொளத்தூர் பெரம்பூர் அம்பத்தூர் அமைந்தகரை ஆகிய இடங்கள் அதிக மழை பொழிவு எதிர்க்கொண்டதாக வானிலை மயம் தகவன் பசும்புன் முத்துராமலிங்க தேவரி 117. ஜெயன்தி மற்றும் 62. குருபுஜகி உட்டி முதல்மைச்சர் முக்கு ச்டாலின் மரியாது பசும்புன்னில் உள்ள தேவரி நினிவிடத்தில் மளர்த்தூபி சிலைக்கு மாலை நிவித்த மரியாதை சொல்றினார் பசும்புன்னில் தேவர் வாழ்ந்த இளம் புதுப்பிக்கப்பட்டு 100 ஆண்டு அலங்கाர வளைவு நினைவிடத்தில் அணயா விளக்கு அமைப்பு தேவர் ஜெயன்தியின் போது பசும்புன்னில் கூட நரிசலை தவிர்க தேவர் அரங்கம் திருக்கப்பட்டுள்ளதாக முதலாமைச்சர் முக்கஸ் டாலின் பேட்டி முதலாமைச்சர் முக்கஸ் டாலின் பேட்டி முதலாமைச்சர் முக்கஸ் டாலி

In [39]:
""" # to run it on CPU!

import yt_dlp

def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }
        ],
        'outtmpl': 'audio.mp3',
        'ffmpeg_location': 'C:/Users/KK/AppData/Local/ffmpeg-essentials_build/bin'  # Explicit path to ffmpeg
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return 'audio.mp3'

# Assuming transcribe_audio_with_whisper is a function that takes an audio path and transcribes it with Whisper
video_url = "https://www.youtube.com/watch?v=RRk9QlCjul0"
audio_path = download_audio(video_url)
transcription = transcribe_audio_with_whisper(audio_path)
print(transcription) """


' # to run it on CPU!\n\nimport yt_dlp\n\ndef download_audio(url):\n    ydl_opts = {\n        \'format\': \'bestaudio/best\',\n        \'postprocessors\': [\n            {\n                \'key\': \'FFmpegExtractAudio\',\n                \'preferredcodec\': \'mp3\',\n                \'preferredquality\': \'192\',\n            }\n        ],\n        \'outtmpl\': \'audio.mp3\',\n        \'ffmpeg_location\': \'C:/Users/KK/AppData/Local/ffmpeg-essentials_build/bin\'  # Explicit path to ffmpeg\n    }\n    with yt_dlp.YoutubeDL(ydl_opts) as ydl:\n        ydl.download([url])\n    return \'audio.mp3\'\n\n# Assuming transcribe_audio_with_whisper is a function that takes an audio path and transcribes it with Whisper\nvideo_url = "https://www.youtube.com/watch?v=RRk9QlCjul0"\naudio_path = download_audio(video_url)\ntranscription = transcribe_audio_with_whisper(audio_path)\nprint(transcription) '